In [ ]:
import ultralytics
from ultralytics import YOLO
import os
import cv2
import time

# Set CUDA Environment Variables
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
# o

CUDA_VERSION = '12.1'
os.environ['PATH'] = '/usr/local/cuda-' + CUDA_VERSION + '/bin:' + os.environ.get('PATH', '')
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-' + CUDA_VERSION + '/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')


# Change these parameters to fit your needs
EPOCHS = 25
IMG_SIZE = 512
BATCH_SIZE = 16
NAME = "NEW_ONECLASS_NEWLABEL_real_epoch" + str(EPOCHS) + "_img" + str(IMG_SIZE) + "_batch" + str(BATCH_SIZE)

# Whether or not to use ray tune for hyperparameter sweep / tuning
USE_RAY_TUNE = False
# Number of iterations for hyperparameter sweep / tuning
TUNE_ITERS = 100

CURR_DIR = os.getcwd()
WORKSPACE_DIR = os.path.dirname(CURR_DIR)
DATASETS_DIR = "../data/data_" + NAME + "/"

# Flag to resume training from a previous checkpoint (false if training from scratch)
RESUME_TRAINING = False

# Path to trained model weights
MODELS_PATH = WORKSPACE_DIR + 'models/'
MODEL_PATH = MODELS_PATH + 'yolov8_fine_tune.pt'

# This line prevents the Kernel from crashing when running model.train() which calls a plotting function
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Check System Information

In [ ]:
ultralytics.utils.checks.collect_system_info()

# Create Training/Validation/Test Folders

In [ ]:
import random
import shutil

# create directories to extract data from
train_dir = '../inputs/train/sim2real_and_real/'
test_dir = '../inputs/test/real_march_data/'
print("Extracting train/validation images from:", train_dir)
print("Extracting test images from:", test_dir)
print()


# creating training/validation data directories
train_images_labels = []
for train_folder in os.listdir(train_dir):
    full_path = os.path.join(train_dir, train_folder)
    for img_file in os.listdir(full_path + "/images/"):
        train_images_labels.append([full_path + "/images/" + img_file, full_path + "/labels/" + img_file[:-4] + ".txt", train_folder])

# sort images by filename
train_images_labels = sorted(train_images_labels, key = lambda x: x[0])

# shuffle images deterministically with seed
random.seed(42)
random.shuffle(train_images_labels)

# split 80% training, 20% validation
training_inputs = train_images_labels[len(train_images_labels) // 5:]
valid_inputs = train_images_labels[:len(train_images_labels) // 5]

# create new directories, in format that YOLOv8 needs
if not os.path.exists("../data/"):
    os.mkdir("../data/")
if os.path.exists(DATASETS_DIR):
    print("Deleting and recreating", DATASETS_DIR, "folder...")
    shutil.rmtree(DATASETS_DIR)
os.mkdir(DATASETS_DIR)
os.mkdir(DATASETS_DIR + "train/")
os.mkdir(DATASETS_DIR + "train/images/")
os.mkdir(DATASETS_DIR + "train/labels/")
os.mkdir(DATASETS_DIR + "val/")
os.mkdir(DATASETS_DIR + "val/images/")
os.mkdir(DATASETS_DIR + "val/labels/")
os.mkdir(DATASETS_DIR + "test/")
os.mkdir(DATASETS_DIR + "test/images/")
os.mkdir(DATASETS_DIR + "test/labels/")

# copy files over into train/validation directories
for img, label, train_folder in training_inputs:
    shutil.copy(img, DATASETS_DIR + "train/images/" + train_folder + "_" + img.split("/")[-1])
    shutil.copy(label, DATASETS_DIR + "train/labels/" + train_folder + "_" + label.split("/")[-1])
for img, label, valid_folder in valid_inputs:
    shutil.copy(img, DATASETS_DIR + "val/images/" + valid_folder + "_" + img.split("/")[-1])
    shutil.copy(label, DATASETS_DIR + "val/labels/" + valid_folder + "_" + label.split("/")[-1])

# create test data directories
test_images_labels = []
for test_folder in os.listdir(test_dir):
    full_path = os.path.join(test_dir, test_folder)
    for img_file in os.listdir(full_path + "/images/"):
        test_images_labels.append([full_path + "/images/" + img_file, full_path + "/labels/" + img_file[:-4] + ".txt", test_folder])

# sort images by filename
test_inputs = sorted(test_images_labels, key = lambda x: x[0])

# copy files over into test directories
for img, label, test_folder in test_images_labels:
    shutil.copy(img, DATASETS_DIR + "test/images/" + test_folder + "_" + img.split("/")[-1])
    shutil.copy(label, DATASETS_DIR + "test/labels/" + test_folder + "_" + label.split("/")[-1])

# copy over data.yaml file from root directory
shutil.copy("../data.yaml", DATASETS_DIR)
print("Copied over 'data.yaml' file\n")

print("Finished creating directories for YOLOv8 training pipeline")

print("Training on", len(training_inputs), "images")
print("Validating on", len(valid_inputs), "images")
print("Testing on", len(test_inputs), "images")

# Train Model

In [ ]:
# Load yolov8 nano segmentation model
if RESUME_TRAINING:
    # TODO: Change this to the path of the model you want to resume training from
    model = YOLO('yolov8n-seg.pt', resume=False)
# Load yolov8 nano segmentation model
else:
    model = YOLO('yolov8n-seg.pt')

# Find dataset images
CURR_DATASET = DATASETS_DIR # Change this to the dataset you want to train on
TRAIN_DATA = CURR_DATASET + 'data.yaml'
TEST_PATH = CURR_DATASET + 'test/images/'
SAVE_PATH = CURR_DATASET + 'test/annotation_results'
# Make sure the save path exists
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

start_time = time.time()
# By default, the model trains on a single GPU
model.train(data=TRAIN_DATA, epochs=EPOCHS, imgsz=IMG_SIZE, name=NAME, batch=BATCH_SIZE)
end_time = time.time()
training_time = end_time - start_time
print("Time to train: ", training_time)

# Hyperparameter Tuning

In [ ]:
# # # # # Runs a hyperparameter sweep and selects the best hyperparameters
# model.tune(use_ray=USE_RAY_TUNE, iterations=TUNE_ITERS)
# # model.tune(epochs=10, iterations=50, optimizer='AdamW', plots=True, save=True, val=True)

# Test Model

In [ ]:
# load up the best model from training
model = YOLO("runs/segment/" + NAME + "/weights/best.pt")

In [ ]:
import tqdm
def images_to_video(image_filenames, output_file, fps=24):

    # Sort the image files to maintain order
    # for filename in image_filenames:
    #     print(filename, int(filename.split("/")[-1].split("_")[1][:-4]), "\n")
    image_files = sorted(image_filenames, key=lambda filename: int(filename.split("/")[-1].split("_")[-1][:-4]))

    # Get the first image to get dimensions
    first_image = cv2.imread(image_files[0])
    height, width, layers = first_image.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for AVI format
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Iterate through each image and write to the video
    for image_file in tqdm.tqdm(image_files):
        image_path = image_file
        frame = cv2.imread(image_path)
        video.write(frame)

    # Release the VideoWriter object
    video.release()

    print(f"Video saved to: {output_file}")

image_files = []
# Inference fine-tuned model on test images and save results
for file in os.listdir(TEST_PATH):
    file_path = os.path.join(TEST_PATH, file)
    output = model.predict(file_path)
    save_path = os.path.join(SAVE_PATH, file)
    image_files.append(save_path)
    cv2.imwrite(save_path, output[0].plot())

print("Inference on test set complete. Results saved to: ", SAVE_PATH)

# Convert the images to a video
images_to_video(image_files, os.path.join(SAVE_PATH, "0_video.mp4"))


# Save Weights and Export Model

In [ ]:
# model.save(MODEL_PATH) # Save the model as a .pt file
# model.export(save_dir=MODELS_PATH, format='onnx', half=True)

# Generate Test Metrics

In [ ]:
# # load all three models
# model1 = YOLO("/home/ashwatc/Desktop/YOLOv8-Fine-Tune/src/runs/segment/FINAL_FOLDER_RESULTS/1_real_epoch25_img512_batch16/weights/best.pt")
# model2 = YOLO("/home/ashwatc/Desktop/YOLOv8-Fine-Tune/src/runs/segment/FINAL_FOLDER_RESULTS/2_sim_epoch25_img512_batch16/weights/best.pt")
# model3 = YOLO("/home/ashwatc/Desktop/YOLOv8-Fine-Tune/src/runs/segment/FINAL_FOLDER_RESULTS/3_sim_R_epoch25_img512_batch16/weights/best.pt")
# model4 = YOLO("/home/ashwatc/Desktop/YOLOv8-Fine-Tune/src/runs/segment/FINAL_FOLDER_RESULTS/4_sim2real_epoch25_img512_batch16/weights/best.pt")
# model5 = YOLO("/home/ashwatc/Desktop/YOLOv8-Fine-Tune/src/runs/segment/FINAL_FOLDER_RESULTS/5_sim2real_R_epoch25_img512_batch16/weights/best.pt")
# models = [model1, model2, model3, model4, model5]

# # load yaml for data
# data_path = "/home/ashwatc/Desktop/YOLOv8-Fine-Tune/data/MISC/data_for_generating_test_benchmark/data.yaml"


In [ ]:
# from ultralytics import YOLO

# val_results_list = []

# # Load a model
# for model in models:
#     # Run evaluation
#     validation_results = model.val(data=data_path,
#                                 imgsz=512,
#                                 batch=16,
#                                 # conf=0.25,
#                                 # iou=0.5,
#                                 # device='0',
#                                 plots=True)
#     val_results_list.append(validation_results)